## FEM for the clover 

In [6]:
from ngsolve import *
from netgen.read_gmsh import ReadGmsh
from netgen.csg import *
import numpy as np
import meshio
import scipy.special as sc
import scipy.sparse as sp

In [2]:
#create mesh
mesh = Mesh(ReadGmsh("clover_mesh.msh"))

In [5]:
List_of_mats = ['metal', 'vacuum']
hankel = lambda n, k : sc.hankel1(n,k)
dhankel = lambda n, k : sc.h1vp(n,k,1)

In [19]:
def FEM(epsm,order,k,DtN_tol):
    #define epsilon parameter
    epsv=1
    eps_val={'metal' : epsm, 'vacuum' : epsv}
    eps=CoefficientFunction([eps_val[mat] for mat in List_of_mats])
    epsinv_val = {'metal' : 1/epsm, 'vacuum' : 1/epsv}
    epsinv=CoefficientFunction([epsinv_val[mat] for mat in List_of_mats])
    
    #create finite element space
    fes = H1(mesh, order=order, complex=True)
   
    #define incident field
    Inc = CoefficientFunction(exp(1j*k*y))
    diff_Inc_x = Inc.Diff(x)
    diff_Inc_y = Inc.Diff(y)
    IncNormDrv = CoefficientFunction((1/sqrt(x*x+y*y))*(x*diff_Inc_x+y*diff_Inc_y))
    
    #prepare and populate linear/bilinear forms
    u, v = fes.TnT()
    a = BilinearForm(fes)
    f = LinearForm(fes)
    a += (-1*epsinv*(grad(u)*grad(v)))*dx
    a += (k**2*u*v)*dx
    ginc = (1/epsv)*(-1*IncNormDrv)
    f += (ginc*v)*ds('outer')
    
    #create DtN
    dtn1_array= np.zeros([DtN_tol,2*DtN_tol+1],dtype='object')
    dtn_mat=sp.csr_matrix(np.zeros([fes.ndof,fes.ndof],dtype="complex"))
    ulminc_mat=np.zeros([DtN_tol],dtype="complex")
    ulm_mat=np.zeros([DtN_tol],dtype="complex")
            
    rhs_array=np.zeros([fes.ndof],dtype='complex')
    print("Creating dtn matrices")
    
    for l in range(0,DtN_tol):
        ulminc_mat[l] = Integrate(Inc*Conj(exp(1j*l*atan2(y,x))),mesh,definedon=mesh.Boundaries("outer"))
        
        #create matching constant for dtn expansion
        self.const_mat[l]=k*(dhankel(l,self.k*self.R)/hankel(l,self.k*self.R))
                    
        #create vector associated with v - dtn applied to all phi
        
            #self.dtn1_array[l,DtN_tol+m]= LinearForm(self.fes)
            #self.dtn1_array[l,DtN_tol+m] += (v*Conj(NGsolve_Spherical_Harmonic(l,m)))*ds('outer')
            #self.dtn1_array[l,DtN_tol+m].Assemble()
            #Right hand side - dtn applied to incident field
            #constant*u^inc_lm*v_lm
            #self.rhs_array += self.const_mat[l,DtN_tol+m]*self.ulminc_mat[l,DtN_tol+m]*np.conjugate(ngvec_to_vec(self.dtn1_array[l,DtN_tol+m].vec))
                    
            #build vectors
                    
            #dtn_vec1=np.conjugate(ngvec_to_vec(self.dtn1_array[l,DtN_tol+m].vec))
            #dtn_vec2=ngvec_to_vec(self.dtn1_array[l,DtN_tol+m].vec)
                    
            #rows of the column vector correspond to row values of the matrix - we want the i-th 
            #row to be associated with a fixed ϕ over which we compose u as a basis of all ϕ
            #self.dtn_mat += self.const_mat[l,DtN_tol+m]*(sp.csr_matrix(dtn_vec1).T*sp.csr_matrix(dtn_vec2))
            #print("Updated matrix with dtn l=" +str(l)+" m=" + str(m))


In [20]:
FEM(-1.1,2,5,15)

Creating dtn matrices
